## Import libraries

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.data
import nltk
import glob
import os
import re
import string
import pandas as pd

## Compile all .txt file data into a dataframe

In [3]:
# create a list of all the files I want to grab
cleantxt_list = glob.glob("Austin_Transcripts/*_c.txt")

# initiate a dictionary
text_dict = {}

# compile df
date_list, text_list = list(), list()
for filename in cleantxt_list:
    with open(filename, 'r') as f:
        data = f.read()
    date = filename.split("/")[1][:-6]
    for element in data.split('_'):
        text_list.append(element)
        date_list.append(date)

In [4]:
text_dict = {'date': date_list, 'text': text_list}
df = pd.DataFrame(text_dict)

In [6]:
df.head()

,date,text
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...
1,2015-03-26,There are 20 that are signed up for the next ...
2,2015-03-26,Okay The other thing is Im going to do everyt...
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...
4,2015-03-26,Good afternoon Three of us are here today and...


In [7]:
df.shape

198.3

In [9]:
df.to_csv("speakers_df.csv")

In [ ]:
list(text_dict.keys())[:2]

In [ ]:
type(text_dict['2015_01_29'])